# 🌀 Kaleidoscope Logic Exploration Notebook

Interactive exploration of the Binocular Kaleidoscope Logic framework.

## Overview
This notebook allows you to:
1. Generate kaleidoscope patterns with different parameters
2. Visualize phase evolution
3. Explore the parameter space interactively
4. Analyze pattern complexity

## Prerequisites
```bash
pip install numpy matplotlib ipywidgets ipykernel
```

In [ ]:
# Cell 1: Setup and imports
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown
import warnings
warnings.filterwarnings('ignore')

# Try to import our kaleidoscope module
try:
    from src.kaleidoscope import BinocularKaleidoscope
    print("✅ Successfully imported kaleidoscope module")
except ImportError:
    print("⚠️  Could not import src.kaleidoscope")
    print("Running in demo mode with simplified functions...")
    
    # Define simplified version for demo
    class BinocularKaleidoscope:
        def __init__(self):
            self.left_phase = 0.0
            self.right_phase = 0.39
            self.delta_left = 0.08
            self.delta_right = 0.095
            self.symmetry_folds = 6
            
        def rotate(self):
            self.left_phase = (self.left_phase + self.delta_left) % (2*np.pi)
            self.right_phase = (self.right_phase + self.delta_right) % (2*np.pi)
            
        def generate_pattern(self):
            # Generate a simple pattern for demo
            t = np.linspace(0, 2*np.pi, 100)
            
            # Left pattern (blue)
            left_x = np.cos(t + self.left_phase) * 0.8
            left_y = np.sin(t*2 + self.left_phase) * 0.8
            
            # Right pattern (red)
            right_x = np.cos(t*1.5 + self.right_phase) * 0.9
            right_y = np.sin(t*3 + self.right_phase) * 0.9
            
            # Combined pattern (purple)
            combined_x = (left_x + right_x) / 2
            combined_y = (left_y + right_y) / 2
            
            return {
                'left': (left_x, left_y),
                'right': (right_x, right_y),
                'combined': (combined_x, combined_y)
            }

## 2. Initialize Kaleidoscope System

In [ ]:
# Cell 2: Initialize kaleidoscope system
kaleido = BinocularKaleidoscope()

print("Kaleidoscope System Initialized:")
print(f"  Left phase: {kaleido.left_phase:.2f} rad")
print(f"  Right phase: {kaleido.right_phase:.2f} rad")
print(f"  Phase difference: {kaleido.right_phase - kaleido.left_phase:.2f} rad")
print(f"  Symmetry folds: {kaleido.symmetry_folds}")

## 3. Single Frame Generation

In [ ]:
# Cell 3: Generate and display a single frame
def plot_kaleidoscope_frame(left_phase=0.0, right_phase=0.39, style='combined'):
    """Plot a kaleidoscope frame with given phases"""
    
    # Update phases
    kaleido.left_phase = left_phase % (2*np.pi)
    kaleido.right_phase = right_phase % (2*np.pi)
    
    # Generate pattern
    pattern = kaleido.generate_pattern()
    
    # Create figure
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Plot left system
    axes[0].scatter(pattern['left'][0], pattern['left'][1], 
                   c='blue', s=20, alpha=0.7)
    axes[0].set_title(f'Left System\nθ = {left_phase:.2f} rad')
    axes[0].set_xlim(-1.5, 1.5)
    axes[0].set_ylim(-1.5, 1.5)
    axes[0].grid(True, alpha=0.3)
    axes[0].set_aspect('equal')
    
    # Plot right system
    axes[1].scatter(pattern['right'][0], pattern['right'][1], 
                   c='red', s=20, alpha=0.7)
    axes[1].set_title(f'Right System\nθ = {right_phase:.2f} rad')
    axes[1].set_xlim(-1.5, 1.5)
    axes[1].set_ylim(-1.5, 1.5)
    axes[1].grid(True, alpha=0.3)
    axes[1].set_aspect('equal')
    
    # Plot combined system
    axes[2].scatter(pattern['combined'][0], pattern['combined'][1], 
                   c='purple', s=20, alpha=0.8)
    axes[2].set_title(f'Binocular Fusion\nΔθ = {right_phase - left_phase:.2f} rad')
    axes[2].set_xlim(-1.5, 1.5)
    axes[2].set_ylim(-1.5, 1.5)
    axes[2].grid(True, alpha=0.3)
    axes[2].set_aspect('equal')
    
    plt.suptitle(f'Kaleidoscope Logic: Phase Analysis', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Print phase info
    print(f"Phase Analysis:")
    print(f"  Left θ: {left_phase:.3f} rad ({np.degrees(left_phase):.1f}°)")
    print(f"  Right θ: {right_phase:.3f} rad ({np.degrees(right_phase):.1f}°)")
    print(f"  Phase difference: {right_phase - left_phase:.3f} rad")
    print(f"  Phase coherence: {np.cos(right_phase - left_phase):.3f}")

# Interactive widget for single frame
interact(plot_kaleidoscope_frame,
         left_phase=FloatSlider(min=0, max=2*np.pi, step=0.1, value=0.0),
         right_phase=FloatSlider(min=0, max=2*np.pi, step=0.1, value=0.39),
         style=Dropdown(options=['combined', 'separate', 'overlay'], value='combined'))

## 4. Sequence Generation

In [ ]:
# Cell 4: Generate and visualize sequence
def generate_sequence(num_frames=25, delta_left=0.08, delta_right=0.095):
    """Generate and plot a sequence of frames"""
    
    # Reset kaleidoscope
    kaleido.left_phase = 0.0
    kaleido.right_phase = 0.39
    kaleido.delta_left = delta_left
    kaleido.delta_right = delta_right
    
    # Store phase data
    phases = []
    
    for frame in range(num_frames):
        phases.append({
            'frame': frame,
            'left': kaleido.left_phase,
            'right': kaleido.right_phase,
            'diff': kaleido.right_phase - kaleido.left_phase
        })
        kaleido.rotate()
    
    # Convert to arrays for plotting
    frames = [p['frame'] for p in phases]
    left_phases = [p['left'] for p in phases]
    right_phases = [p['right'] for p in phases]
    phase_diffs = [p['diff'] for p in phases]
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    # Phase evolution
    axes[0, 0].plot(frames, left_phases, 'b-', label='Left θ', linewidth=2)
    axes[0, 0].plot(frames, right_phases, 'r-', label='Right θ', linewidth=2)
    axes[0, 0].set_xlabel('Frame')
    axes[0, 0].set_ylabel('Phase (rad)')
    axes[0, 0].set_title('Phase Evolution')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Phase difference
    axes[0, 1].plot(frames, phase_diffs, 'g-', linewidth=2)
    axes[0, 1].fill_between(frames, phase_diffs, alpha=0.2, color='green')
    axes[0, 1].set_xlabel('Frame')
    axes[0, 1].set_ylabel('Phase Difference Δθ (rad)')
    axes[0, 1].set_title('Perceptual Drift')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Phase space trajectory
    axes[1, 0].plot(left_phases, right_phases, 'purple', linewidth=1, alpha=0.5)
    scatter = axes[1, 0].scatter(left_phases, right_phases, c=frames, 
                                cmap='viridis', s=30, alpha=0.8)
    axes[1, 0].set_xlabel('Left Phase θL')
    axes[1, 0].set_ylabel('Right Phase θR')
    axes[1, 0].set_title('Phase Space Trajectory')
    axes[1, 0].grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=axes[1, 0], label='Frame')
    
    # Pattern complexity (simulated)
    complexity = np.abs(np.sin(np.array(left_phases)) * np.cos(np.array(right_phases)))
    axes[1, 1].plot(frames, complexity, 'orange', linewidth=2)
    axes[1, 1].fill_between(frames, complexity, alpha=0.2, color='orange')
    axes[1, 1].set_xlabel('Frame')
    axes[1, 1].set_ylabel('Pattern Complexity')
    axes[1, 1].set_title('Pattern Complexity Evolution')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.suptitle(f'Sequence Analysis: {num_frames} frames, ΔL={delta_left:.3f}, ΔR={delta_right:.3f}', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"Sequence Statistics:")
    print(f"  Frames: {num_frames}")
    print(f"  Initial Δθ: {phase_diffs[0]:.3f} rad")
    print(f"  Final Δθ: {phase_diffs[-1]:.3f} rad")
    print(f"  Δθ per frame: {delta_right - delta_left:.3f} rad")
    print(f"  Max complexity: {complexity.max():.3f} at frame {np.argmax(complexity)}")
    
    return phases

# Interactive sequence generation
interact(generate_sequence,
         num_frames=IntSlider(min=5, max=100, step=5, value=25),
         delta_left=FloatSlider(min=0.01, max=0.5, step=0.01, value=0.08),
         delta_right=FloatSlider(min=0.01, max=0.5, step=0.01, value=0.095))

## 5. Parameter Space Exploration

In [ ]:
# Cell 5: Explore parameter space
def explore_parameter_space(symmetry_folds=6, fragments=9, 
                           color_scheme='philosophical', pattern_type='spiral'):
    """Explore the parameter space of kaleidoscope patterns"""
    
    # Generate sample patterns based on parameters
    np.random.seed(42)
    
    if pattern_type == 'spiral':
        angles = np.linspace(0, 3*np.pi, fragments)
        radii = np.linspace(0.3, 1.2, fragments)
        points = np.column_stack([radii * np.cos(angles), radii * np.sin(angles)])
    elif pattern_type == 'circle':
        angles = np.linspace(0, 2*np.pi, fragments, endpoint=False)
        points = np.column_stack([np.cos(angles), np.sin(angles)]) * 0.8
    else:  # random
        points = np.random.uniform(-1, 1, (fragments, 2))
    
    # Apply symmetry
    symmetric_points = []
    for point in points:
        symmetric_points.append(point)
        for fold in range(1, symmetry_folds):
            angle = fold * (2*np.pi / symmetry_folds)
            cos_a, sin_a = np.cos(angle), np.sin(angle)
            x, y = point
            rot_point = np.array([x * cos_a - y * sin_a, x * sin_a + y * cos_a])
            symmetric_points.append(rot_point)
            symmetric_points.append([-rot_point[0], rot_point[1]])
    
    symmetric_points = np.array(symmetric_points)
    
    # Create colors based on scheme
    if color_scheme == 'philosophical':
        colors = plt.cm.Blues(np.linspace(0.3, 0.9, len(symmetric_points)))
    elif color_scheme == 'complementary':
        colors = plt.cm.RdYlBu(np.linspace(0.2, 0.8, len(symmetric_points)))
    else:  # rainbow
        colors = plt.cm.rainbow(np.linspace(0, 1, len(symmetric_points)))
    
    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Original fragments
    axes[0].scatter(points[:, 0], points[:, 1], 
                   c=plt.cm.Set1(np.arange(len(points)) / len(points)),
                   s=100, edgecolors='black', linewidth=1)
    axes[0].set_title(f'Original Fragments ({fragments} points)')
    axes[0].set_xlim(-1.5, 1.5)
    axes[0].set_ylim(-1.5, 1.5)
    axes[0].grid(True, alpha=0.3)
    axes[0].set_aspect('equal')
    
    # Symmetric pattern
    axes[1].scatter(symmetric_points[:, 0], symmetric_points[:, 1], 
                   c=colors, s=20, alpha=0.7, edgecolors='white', linewidth=0.5)
    axes[1].set_title(f'Symmetric Pattern ({symmetry_folds}-fold symmetry)')
    axes[1].set_xlim(-2, 2)
    axes[1].set_ylim(-2, 2)
    axes[1].grid(True, alpha=0.3)
    axes[1].set_aspect('equal')
    
    plt.suptitle(f'Parameter Exploration: {pattern_type} pattern, {color_scheme} colors', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"Parameter Analysis:")
    print(f"  Symmetry folds: {symmetry_folds}")
    print(f"  Fragments: {fragments}")
    print(f"  Color scheme: {color_scheme}")
    print(f"  Pattern type: {pattern_type}")
    print(f"  Total points: {len(symmetric_points)}")
    print(f"  Pattern density: {len(symmetric_points) / (4*4):.1f} points/unit²")

# Interactive parameter exploration
interact(explore_parameter_space,
         symmetry_folds=IntSlider(min=3, max=12, step=1, value=6),
         fragments=IntSlider(min=3, max=20, step=1, value=9),
         color_scheme=Dropdown(options=['philosophical', 'complementary', 'rainbow'], 
                               value='philosophical'),
         pattern_type=Dropdown(options=['spiral', 'circle', 'random'], 
                               value='spiral'))

## 6. Export and Save

In [ ]:
# Cell 6: Export functions
import json
from datetime import datetime

def save_experiment(name, parameters, results):
    """Save experiment results to JSON"""
    
    experiment = {
        'name': name,
        'timestamp': datetime.now().isoformat(),
        'parameters': parameters,
        'results': results
    }
    
    filename = f"experiment_{name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    with open(filename, 'w') as f:
        json.dump(experiment, f, indent=2, default=str)
    
    print(f"✅ Experiment saved to {filename}")
    return filename

def load_experiment(filename):
    """Load experiment from JSON"""
    with open(filename, 'r') as f:
        experiment = json.load(f)
    
    print(f"📊 Loaded experiment: {experiment['name']}")
    print(f"  Date: {experiment['timestamp']}")
    print(f"  Parameters: {len(experiment['parameters'])} parameters")
    
    return experiment

# Example usage
print("Export functions ready!")
print("Use save_experiment() to save your exploration results.")
print("Use load_experiment() to load previous experiments.")

---

## Summary

This notebook provides interactive exploration of:

1. **Single frame generation** with adjustable phases
2. **Sequence analysis** with phase evolution visualization
3. **Parameter space exploration** for symmetry and patterns
4. **Export functionality** for saving experiments

To use with the full kaleidoscope module:
```python
from src.kaleidoscope import BinocularKaleidoscope
kaleido = BinocularKaleidoscope()
```